# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [11]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [12]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

# Cargar dataset
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()



,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [14]:
# --- BLOQUE 1 ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# Cargar datos
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")

# Limpieza rápida (mismo criterio del lab previo)
spaceship = spaceship.dropna()

# Cabin -> solo la letra (deck)
spaceship["Cabin"] = spaceship["Cabin"].apply(lambda x: x[0])

# Quitar columnas poco útiles
spaceship = spaceship.drop(columns=["PassengerId", "Name"])

# One-hot encoding (categóricas)
spaceship = pd.get_dummies(spaceship, drop_first=True)

# X / y y split
X = spaceship.drop("Transported", axis=1)
y = spaceship["Transported"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape



((5284, 19), (1322, 19))

- Evaluate your model

In [15]:
# --- BLOQUE 2 ---
# Pipeline base: Escalado + Selección de características + KNN (valores por defecto)
base_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("select", SelectKBest(score_func=mutual_info_classif, k="all")),  # de inicio no filtramos
    ("knn", KNeighborsClassifier())
])

# Entrenar y evaluar baseline
base_pipe.fit(X_train, y_train)
y_pred_base = base_pipe.predict(X_test)

print("Baseline Accuracy:", accuracy_score(y_test, y_pred_base))
print(classification_report(y_test, y_pred_base))


Baseline Accuracy: 0.7639939485627837
              precision    recall  f1-score   support

       False       0.77      0.75      0.76       656
        True       0.76      0.78      0.77       666

    accuracy                           0.76      1322
   macro avg       0.76      0.76      0.76      1322
weighted avg       0.76      0.76      0.76      1322



**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [16]:
# --- BLOQUE 3 ---
# Probamos también cuántas features quedarnos con SelectKBest
k_values = [min(10, X_train.shape[1]), min(15, X_train.shape[1]), "all"]

param_grid = {
    "select__k": k_values,
    "knn__n_neighbors": [3, 5, 7, 9, 11, 15],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2],               # 1=Manhattan, 2=Euclidiana
    "knn__leaf_size": [15, 30, 45]
}
param_grid


{'select__k': [10, 15, 'all'],
 'knn__n_neighbors': [3, 5, 7, 9, 11, 15],
 'knn__weights': ['uniform', 'distance'],
 'knn__p': [1, 2],
 'knn__leaf_size': [15, 30, 45]}

- Run Grid Search

In [17]:
grid = GridSearchCV(
    estimator=base_pipe,
    param_grid=param_grid,
    scoring="accuracy",
    cv=5,
    n_jobs=-1,
    verbose=0
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV score:", grid.best_score_)


Best params: {'knn__leaf_size': 45, 'knn__n_neighbors': 9, 'knn__p': 2, 'knn__weights': 'uniform', 'select__k': 10}
Best CV score: 0.7905012757661765


- Evaluate your model

In [18]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

print("Test Accuracy (tuned):", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# (Opcional) Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
cm


Test Accuracy (tuned): 0.7866868381240545
              precision    recall  f1-score   support

       False       0.79      0.77      0.78       656
        True       0.78      0.80      0.79       666

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322



array([[508, 148],
       [134, 532]])

In [19]:
acc_base = accuracy_score(y_test, y_pred_base)
acc_tuned = accuracy_score(y_test, y_pred)

print(f"Baseline Accuracy: {acc_base:.4f}")
print(f"Tuned    Accuracy: {acc_tuned:.4f}")
improvement = acc_tuned - acc_base
print(f"Mejora absoluta:  {improvement:.4f}")

Baseline Accuracy: 0.7640
Tuned    Accuracy: 0.7867
Mejora absoluta:  0.0227
